This notebook contains the Sage scripts used to compute the probability bounds from the paper:

*Analyzing the Real-Work Security of the Algorand Blockchain*

to be published at ACM CCS 2023.

Eprint: https://eprint.iacr.org/2023/1344



**WARNING**: This is a [Sage](https://www.sagemath.org/) Jupyter notebook. This is not a pure Python Jupyter notebook.

Open using 

```bash
sage -n jupyter proba.ipynb
```

In [1]:
from scipy.stats import poisson, binom, hypergeom, norm
import numpy as np
import collections

## Parameters

This section just defines the various parameters we use.

In [2]:
Step = collections.namedtuple("Step", ["name", "E", "t"])
# E = size of committee
# t = threshold

steps = [
    Step("soft", 2990, 2267),
    Step("cert", 1500, 1112),
    Step("next", 5000, 3838),
    Step("late", 500, 320),
    Step("redo", 2400, 1768),
    Step("down", 6000, 4560),
    Step("prop", 20, 1)
]

named_steps = {step.name: step for step in steps}

alpha = 0.2 # probability to be malicious
p = 1 - alpha

## Safety/Validity Failures (Theorem 6.1 and Lemma A.20) - Events A.1-A.6

### Event A.1

This uses Lemma A.19.

In [3]:
Nmin = 1e12


def poi(x, k):
    return x**k / (factorial(k) * exp(x))

def step2failure_formal(E2, T2):
    """    
    Return the probability of failure and its log in base 2
    for the step 2 / soft vote step (expected size E2, quorum/threshold T2)
    That is the probability the adversary control X parties on the committee
    and there are Y honest parties on the committee so that:
    2 X + Y >= 2 * T2
    """
    
    p0 = E2/Nmin
    lamY = alpha*E2 + p0
    lamZ = E2*(1-alpha) + p0
    NN = int(1.5e3) # a bound for our sum, we use this value
    # we chose this value because we remark 
    # that the missing terms of the computation
    # are smaller than sum(poi(E2*alpha + E2/Nmin, i)) for i=NN,...
    # which is poisson(E2*alpha + E2/Nmin).sf(NN-1)
    # which is negligible:
    negl = poisson(lamY).sf(NN-1)
    assert(log(float(negl),2).n() < -256)
    # we anyway add those terms later to be on the safe side

    def too_many_honest_prob(num_mal):
        return poi(lamY, num_mal) * poisson(lamZ).sf(2*T2-2*num_mal-1)

    pf = sum(too_many_honest_prob(i) for i in (0..NN)) + negl
    pflog2 = log(float(pf), 2).n()
    
    return pf, pflog2


print(f"Probability of safety failure for soft-vote committee:")

step = steps[0]
name = step.name
assert name == "soft"
E2 = step.E
T2 = step.t
pf, pflog2 = step2failure_formal(E2, T2)
print(f"  {name} E={E2:4d} t={T2:4d}: 2^{pflog2:.1f}")

Probability of safety failure for soft-vote committee:
  soft E=2990 t=2267: 2^-128.2


### Events A.2 - A.6

This uses Lemma A.20. (We directly use Chernoff here as the bounds are already good enough this way.)

In [4]:
for step in steps[1:-1]:
    # using the new Chernoff bound from the paper,
    # e^{-(\alpha E-Q)^2/(\alpha E+Q)}  --- Q = T
    pf = exp(-(alpha * step.E - step.t)**2 / (alpha * step.E + step.t))
    pflog2 = log(pf, 2)
    pf250log2 = log(250 * pf, 2)
    pf250s = f" (250 iters union bound: 2^{pf250log2:.3f})" if step.name == "next" else ""
    print(f"  {step.name} E={step.E:4d} t={step.t:4d}: {pf:.5f}=2^{pflog2:.3f} {pf250s}")

  cert E=1500 t=1112: 0.00000=2^-673.677 
  next E=5000 t=3838: 0.00000=2^-2401.781  (250 iters union bound: 2^-2393.816)
  late E= 500 t= 320: 0.00000=2^-166.253 
  redo E=2400 t=1768: 0.00000=2^-1064.658 
  down E=6000 t=4560: 0.00000=2^-2827.682 


## Liveness Failures (Theorem 6.2, Lemma A.23, and Corollary A.24) - Events B.1-B.5

This uses Lemma A.22.

In [5]:
def live_fail_bound(step):
    lambda_ = step.E * p
    pf = poisson(lambda_).cdf(step.t-1) * exp(((step.t-1)*lambda_*2+step.t-1)/ (2*minN*p))
    pflog2 = log(float(pf), 2).n()
    return pf, pflog2

minN = 1e12
print(f"Proven bounds of probability that not enough honest parties in committees (assuming N>{minN}):")
for step in steps:
    pf, pflog2 = live_fail_bound(step)
    print(f"  {step.name} E={step.E:4d} t={step.t:4d}: {pf:.5f}=2^{pflog2:.1f}")
    
print("B1.")
c = 60
pf_PV, _ = live_fail_bound(named_steps["prop"])
pf_SV, _ = live_fail_bound(named_steps["soft"])
pf_CV, _ = live_fail_bound(named_steps["cert"])
pf_gc = (1 - (1 - alpha) * (1 - pf_PV)) + pf_SV + pf_CV
pflog2_gc = log(float(pf_gc), 2).n() * c
print(f"  2^{pflog2_gc}")

print("B2.")
c_ = 15
pf_NV, _ = live_fail_bound(named_steps["next"])
pf_nv = c * (pf_NV**(c_ + 1) + (c_ + 1) * pf_NV**(c_) * (1 - pf_NV) + (c_ * (c_ + 1)/2) * pf_NV**(c_-1) * (1 - pf_NV)**2)
pflog2_nv = log(float(pf_nv), 2).n()
print(f"  2^{pflog2_nv}")

Proven bounds of probability that not enough honest parties in committees (assuming N>1.00000000000000e12):
  soft E=2990 t=2267: 0.00486=2^-7.7
  cert E=1500 t=1112: 0.00491=2^-7.7
  next E=5000 t=3838: 0.00488=2^-7.7
  late E= 500 t= 320: 0.00002=2^-16.0
  redo E=2400 t=1768: 0.00021=2^-12.2
  down E=6000 t=4560: 0.00023=2^-12.1
  prop E=  20 t=   1: 0.00000=2^-23.1
B1.
  2^-135.187816217542
B2.
  2^-94.7139918092363


## Inconsistency Failures (Theorem 6.4) - Events C.1-C.4

This uses Lemma 6.3.

In [6]:
def red_blue_win_bound(Tr, Er, Tb, Eb):
    """
    Compute the bound for the winning probability of the red blue game from Lemma 6.3
    """
    gamma = Tr / Er + Tb / Eb
    delta = (1+alpha)/gamma #  = lambda in https://arxiv.org/pdf/1709.08157.pdf
    assert(delta <= 1)
    u = var('u') # takes the place of 't' in paper
    lnp = u * (1 + alpha) - Tr * log(1+u/Er) - Tb * log(1+u/Eb)
    log2p = lnp / ln(2)
    (min_log2p, min_log2p_x) = find_local_minimum(log2p, 0, 2000)
    return min_log2p

def print_red_blue(step_r_name, step_b_name):
    """
    Print computation of red blue with the parameters from the steps named as argument
    """
    step_r = named_steps[step_r_name]
    step_b = named_steps[step_b_name]
    log2p = red_blue_win_bound(step_r.t, step_r.E, step_b.t, step_b.E)
    log2p250 = log(250, 2).n() + log2p
    log2p250s = f"(250 iters union bound: {log2p250})" if step_b_name == "next" else ""
    print(f"  log2: {log2p}  {log2p250s}")

print("C1.")
print_red_blue("cert", "next")
print("C2.")
print_red_blue("cert", "down")
print("C3.")
print_red_blue("soft", "next")
print("C4")
print_red_blue("soft", "redo")

C1.
  log2: -128.99314946080835  (250 iters union bound: -121.027365176146)
C2.
  log2: -128.926700863214  
C3.
  log2: -222.09420642539348  (250 iters union bound: -214.128422140731)
C4
  log2: -129.39867875776855  
